### **Purpose: ** 

In [41]:
player = 'Nick van den Berg'
team = 'eu'

In [42]:
import pickle
import pandas as pd

path = '//DREW/Users/andrew/Desktop/mosconi/'
dframe = pickle.load(open(path+'pkl/allyears_clean_locs','rb'))
if 'a' in team.lower():
    df = dframe[dframe['American_player'].str.contains(player)]
    df['Europe_lost'] = ~ df['Europe_won']
    df = df[['Format', 'European_player', 'European_score', 'American_score',
       'American_player', 'Europe_lost','Europe_won']]
    df.columns = ['Format', 'European_player', 'European_score', 'American_score',
       'American_player', 'America_won', 'America_lost']
else:
    df = dframe[dframe['European_player'].str.contains(player)]
    df['Europe_lost'] = ~ df['Europe_won']

In [44]:
def stats(df):
    try:
        if 'a' in team.lower():
            win = df['America_won'].sum()
            loss = df['America_lost'].sum()
        else:
            win = df['Europe_won'].sum()
            loss = df['Europe_lost'].sum()
        mp= win+loss
        wl = 'Win-loss: {}-{}'.format(win,loss)
        pct = round(win/(win+loss)*100,0)
        return('Matches Played: {}'.format(mp),wl,'Pct: {}'.format(pct))
    except:
        return('No matches to display')

### Lets have a quick look at his overall stats by checking out his singles, doubles, and team stats.

In [45]:
sing = df[df['Format']=='Singles']
dub = df[df['Format']=='Doubles']
teams = df[df['Format']=='Teams']
triples = df[df['Format']=='Triples'] 

In [46]:
'Overall: {}'.format(stats(df))
'Singles: {}'.format(stats(sing))
'Doubles: {}'.format(stats(dub))
'Teams:   {}'.format(stats(teams))
'Triples: {}'.format(stats(triples))

"Overall: ('Matches Played: 40', 'Win-loss: 15-25', 'Pct: 38.0')"

"Singles: ('Matches Played: 14', 'Win-loss: 6-8', 'Pct: 43.0')"

"Doubles: ('Matches Played: 18', 'Win-loss: 7-11', 'Pct: 39.0')"

"Teams:   ('Matches Played: 6', 'Win-loss: 2-4', 'Pct: 33.0')"

"Triples: ('Matches Played: 1', 'Win-loss: 0-1', 'Pct: 0.0')"

## So he plays *significantly*/*slightly* better in singles/doubles matches. 
### Let's see if he plays better with certain partners. 

In [47]:

if 'a' in team.lower():
    dub = dub[['European_player','American_player','America_won','America_lost']]
    dub['Partner']=dub['American_player'].str.replace(player,'')
    dub['Partner']=dub['Partner'].str.replace('&','')
    dub['Partner']=dub['Partner'].str.strip()
    dub.groupby('Partner').sum().sort_values(['America_won','America_lost'],ascending=False)    
else:
    dub = dub[['European_player','American_player','Europe_won','Europe_lost']]
    dub['Partner']=dub['European_player'].str.replace(player,'')
    dub['Partner']=dub['Partner'].str.replace('&','')
    dub['Partner']=dub['Partner'].str.strip()
    dub.groupby('Partner').sum().sort_values(['Europe_won','Europe_lost'],ascending=False)

,Europe_won,Europe_lost
Partner,,
Steve Davis,2.0,3.0
Niels Feijen,1.0,2.0
Imran Majid,1.0,1.0
Karl Boyes,1.0,0.0
Mika Immonen,1.0,0.0
Nick Ekonomopoulos,1.0,0.0
Chris Melling,0.0,2.0
Darren Appleton,0.0,2.0
Ralf Souquet,0.0,1.0


### Looks like he plays best when partnered with -- and plays bad with --.

More questions to answer:
1. Does he play better or worse against certain opponents?
2. Has his performance changed over the years?
3. Is performance affected by location (home vs away)?

## Let's see if he performs better or worse against certain opponents in singles matches.

In [48]:
if 'a' in team.lower():
    sing.groupby('European_player').sum().sort_values('America_won',ascending = False)
else:
    sing.groupby('American_player').sum().sort_values('Europe_won',ascending = False)


,Europe_won,Europe_lost
American_player,,
Corey Deuel,1.0,2.0
Earl Strickland,1.0,1.0
Johnny Archer,1.0,1.0
Mike Davis,1.0,0.0
Mike Dechaine,1.0,0.0
Tony Robles,1.0,0.0
Dennis Hatch,0.0,1.0
Shane Van Boening,0.0,3.0


## Apparently, he does well against ----, and does poorly against ----. 
## Let's see what his record was for each year.

In [49]:
lst =df.index.values.tolist() 
years = set([])
for item in lst:
    years.add(item[0])
years=list(years)

In [50]:
yearly=[]
for year in years:
    if 'a' in team:
        win = df.loc[year]['America_won'].sum()
        loss = df.loc[year]['America_lost'].sum()
    else:
        win = df.loc[year]['Europe_won'].sum()
        loss = df.loc[year]['Europe_lost'].sum()
    yearly.append((win,loss))

In [51]:
yr= {}
for k,v in zip(years,yearly):
    yr[k]=v

In [52]:
yr

{2002: (3, 2),
 2003: (1, 3),
 2006: (3, 5),
 2010: (2, 4),
 2011: (2, 4),
 2012: (3, 3),
 2015: (1, 4)}

### So his worst years were: ---- and his best years were: ----. Let's dig a bit deeper to see how big of a factor location is for his performance.

In [53]:
locs = pickle.load(open(path+'pkl/dloc','rb'))

In [54]:
hm = []
aw = []

for year in years:
    if 'USA' in locs[year]:
        if 'a' in team.lower():
            hm.append(year)
        else:
            aw.append(year)
    else:
        if 'a' in team.lower():
            aw.append(year)
        else:
            hm.append(year)
away = df.loc[aw]
home = df.loc[hm]

In [55]:
'Played home {} times, away {} times'.format(len(hm),len(aw))

'Home Games: {}'.format(stats(home))

'Away Games: {}'.format(stats(away))

'Played home 4 times, away 3 times'

"Home Games: ('Matches Played: 25', 'Win-loss: 11-14', 'Pct: 44.0')"

"Away Games: ('Matches Played: 15', 'Win-loss: 4-11', 'Pct: 27.0')"

### So he plays better ----.

### Let's drill down a bit further to parse out his singles and doubles matches for both home and away 

In [56]:
hsing = stats(home[home['Format']=='Singles'])

In [57]:
hdub = stats(home[home['Format']=='Doubles'])

In [58]:
asing = stats(away[away['Format']=='Singles'])

In [59]:
adub = stats(away[away['Format']=='Doubles'])

In [60]:
'Home Singles: {}'.format(hsing)
'Home Doubles: {}'.format(hdub)
'Away Singles: {}'.format(asing)
'Away Doubles: {}'.format(adub)

"Home Singles: ('Matches Played: 8', 'Win-loss: 4-4', 'Pct: 50.0')"

"Home Doubles: ('Matches Played: 11', 'Win-loss: 6-5', 'Pct: 55.0')"

"Away Singles: ('Matches Played: 6', 'Win-loss: 2-4', 'Pct: 33.0')"

"Away Doubles: ('Matches Played: 7', 'Win-loss: 1-6', 'Pct: 14.0')"

### ---------------------------------------